In [1]:
import pandas as pd
import re

from nltk import PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

col = ['target', 'id', 'date', 'flag', 'user', 'text']

df = pd.read_csv(r'C:\Users\eliya\PycharmProjects\FinalProject\dataset.csv', header = None, names = col,  encoding='latin-1')

# df.fillna(value='', inplace=True)

df.dropna()

# df = df[['text','target']]
df = df.sample(n=25000)

# Handle Categories variable
df['target'] = df['target'].replace(4,1)


df['text'].head()

1308967    loosing my scarface virginity now.. movie is r...
278429     @lilrainbow89 Aww dont gooo.. hope your around...
909137     @EricaLeigh777 $10 for evening, $5 for matinee...
453056                  @Sophcornwell  it was a shock tactic
286342     @electrcspacegrl  I'm sorry!  If I was in the ...
Name: text, dtype: object

In [2]:

def get_part_of_day(h):
        if   5 <= h <= 11:
            return 'morning'
            # return '0'
        elif 12 <= h <= 17:
            return "afternoon"
            # return "1"
        elif 18 <= h <= 22:
            return 'evening'
            # return '2'
        else:
            return "night"
            # return "3"


day = []
month = []
dayInMonth = []
timeOfTweet = []
timePeriod = []
year = []

for d in df['date']:
    sub = d.split(' ')
    # day.append(dayInMonth(sub[0]))
    day.append(sub[0])
    # month.append(sub[1])
    dayInMonth.append(sub[2])
    # timeOfTweet.append(sub[3])
    timePeriod.append(get_part_of_day(int(sub[3][:2])))

    # time zones is only pdt
    # timeZone.append(sub[4])

    # year is only 2009
    # year.append(sub[5])



data = pd.DataFrame()

data['day'] = day
# data['month'] = month
data['dayInMonth'] = dayInMonth
# data['time'] = time
data['partOfDay'] = timePeriod

data = pd.get_dummies(data, columns=['day','partOfDay'])



In [3]:
data['target'] = df['target'].values
data['text'] = df['text'].values

df = data

data.head()

,dayInMonth,day_Fri,day_Mon,day_Sat,day_Sun,day_Thu,day_Tue,day_Wed,partOfDay_afternoon,partOfDay_evening,partOfDay_morning,partOfDay_night,target,text
0,02,0,0,0,0,0,1,0,0,1,0,0,1,loosing my scarface virginity now.. movie is r...
1,01,0,1,0,0,0,0,0,0,0,1,0,0,@lilrainbow89 Aww dont gooo.. hope your around...
2,09,0,0,1,0,0,0,0,0,1,0,0,1,"@EricaLeigh777 $10 for evening, $5 for matinee..."
3,07,0,0,0,1,0,0,0,1,0,0,0,0,@Sophcornwell it was a shock tactic
4,01,0,1,0,0,0,0,0,0,0,1,0,0,@electrcspacegrl I'm sorry! If I was in the ...


In [4]:
print("starting text clean")


ps = PorterStemmer()
sb = SnowballStemmer(language='english')


!pip install langdetect
import nltk
nltk.download('stopwords')
# from langdetect import detect

all_stopwords = stopwords.words('english')

corpus = []
corpus1 = []



for sen in df['text']:
    # remove hashtags
    sen = re.sub("(@[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", ' ', sen)
#     sen = re.sub('(@|#|&)+([a-zA-Z0-9_]+)', '', sen)
    # remove weird chars
    sen = re.sub('[^a-zA-z\'\"]+', ' ', sen)
    # remove urls
    sen = re.sub(r'\$\w*', '', sen)
    # remove old style retweet text "RT"
    sen = re.sub(r'^RT[\s]+', '', sen)
    # remove hyperlinks
    sen = re.sub(r'https?:\/\/.*[\r\n]*', '', sen)
    # only removing the hash # sign from the word
    sen = re.sub(r'#', '', sen)
#     try:
#         if detect(sen) != 'en':
#             sen = None
#             continue
#     except:
#         continue
#         print("This row throws and error:", sen)

    

    sen = sen.lower()
    sen = sen.split()



    
    
    text1 = [ps.stem(word) for word in sen if not word in set(all_stopwords)]
#     text1 = [sb.stem(word) for word in sen if not word in set(all_stopwords)]
    
#     text = ' '.join(sen)
    text1 = ' '.join(text1)
    
    
#     corpus.append(text)
    corpus1.append(text1)



starting text clean


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eliya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# labels = df['target'].values
# text = df['text'].values

# df['text'].head()
# d1 = pd.DataFrame({'col':corpus1})


# d1['target'] = df['target']
# d1.head()


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vector = vectorizer.fit_transform(corpus1)

In [7]:
print(type(vector))
words = pd.DataFrame.sparse.from_spmatrix(vector)
print(type(words))
df = pd.concat([df, words], axis=1, join='inner')


df = df.drop('text',axis=1)

df[df.columns[15:]] = df.iloc[:, 15:].astype('float16')


<class 'scipy.sparse.csr.csr_matrix'>
<class 'pandas.core.frame.DataFrame'>


In [8]:
# text_ext = Pipeline([('tfidf',TfidfVectorizer()),('clf',MLPClassifier())])
# text_ext1 = Pipeline([('tfidf',TfidfVectorizer()),('clf',MLPClassifier())])



In [9]:
y = df.target
X = df.drop('target', axis=1)
X1 = pd.DataFrame.sparse.from_spmatrix(vector)

X.head()

,dayInMonth,day_Fri,day_Mon,day_Sat,day_Sun,day_Thu,day_Tue,day_Wed,partOfDay_afternoon,partOfDay_evening,...,17956,17957,17958,17959,17960,17961,17962,17963,17964,17965
0,02,0,0,0,0,0,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,09,0,0,1,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,07,0,0,0,1,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
X1.head()

,0,1,2,3,4,5,6,7,8,9,...,17956,17957,17958,17959,17960,17961,17962,17963,17964,17965
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##  Only text vectorization

In [11]:
print("starting splitting data")

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.3, random_state = 42)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y,test_size = 0.3, random_state = 42)


starting splitting data


In [12]:
# models

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier


DecTr = DecisionTreeClassifier()
AdaBoost = AdaBoostClassifier()
Knn = KNeighborsClassifier()

DecTr1 = DecisionTreeClassifier()
AdaBoost1 = AdaBoostClassifier()
Knn1 = KNeighborsClassifier()


In [13]:
print("fitting extended data")
DecTr.fit(X_train,y_train)
AdaBoost.fit(X_train,y_train)
Knn.fit(X_train,y_train)

fitting extended data


C:\Users\eliya\anaconda3\lib\site-packages\sklearn\utils\validation.py:572: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\eliya\anaconda3\lib\site-packages\sklearn\utils\validation.py:572: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\eliya\anaconda3\lib\site-packages\sklearn\utils\validation.py:572: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\eliya\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


KNeighborsClassifier()

In [14]:
print("fitting only text")
DecTr1.fit(X_train1,y_train1)
AdaBoost1.fit(X_train1,y_train1)
Knn1.fit(X_train1,y_train1)

fitting only text


KNeighborsClassifier()

In [15]:
print("predicting extended")
predictionsa = DecTr.predict(X_test)
predictionsb = AdaBoost.predict(X_test)
predictionsc = Knn.predict(X_test)

predicting extended


C:\Users\eliya\anaconda3\lib\site-packages\sklearn\utils\validation.py:572: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\eliya\anaconda3\lib\site-packages\sklearn\utils\validation.py:572: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\eliya\anaconda3\lib\site-packages\sklearn\utils\validation.py:572: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\eliya\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [16]:
print("predicting original")
predictions1a = DecTr1.predict(X_test1)
predictions1b = AdaBoost1.predict(X_test1)
predictions1c = Knn1.predict(X_test1)

predicting original


In [17]:
print("Decision tree\n")

print("extended:")
print(confusion_matrix(y_test,predictionsa))
print(classification_report(y_test,predictionsa))
print(accuracy_score(y_test,predictionsa))

print("original:")
print(confusion_matrix(y_test1,predictions1a))
print(classification_report(y_test1,predictions1a))
print(accuracy_score(y_test1,predictions1a))

Decision tree

extended:
[[2726 1011]
 [1223 2540]]
              precision    recall  f1-score   support

           0       0.69      0.73      0.71      3737
           1       0.72      0.67      0.69      3763

    accuracy                           0.70      7500
   macro avg       0.70      0.70      0.70      7500
weighted avg       0.70      0.70      0.70      7500

0.7021333333333334
original:
[[2623 1114]
 [1300 2463]]
              precision    recall  f1-score   support

           0       0.67      0.70      0.68      3737
           1       0.69      0.65      0.67      3763

    accuracy                           0.68      7500
   macro avg       0.68      0.68      0.68      7500
weighted avg       0.68      0.68      0.68      7500

0.6781333333333334


In [18]:
print("Adaboost\n")

print("extended:")
print(confusion_matrix(y_test,predictionsb))
print(classification_report(y_test,predictionsb))
print(accuracy_score(y_test,predictionsb))

print("original:")
print(confusion_matrix(y_test1,predictions1b))
print(classification_report(y_test1,predictions1b))
print(accuracy_score(y_test1,predictions1b))

Adaboost

extended:
[[2444 1293]
 [ 879 2884]]
              precision    recall  f1-score   support

           0       0.74      0.65      0.69      3737
           1       0.69      0.77      0.73      3763

    accuracy                           0.71      7500
   macro avg       0.71      0.71      0.71      7500
weighted avg       0.71      0.71      0.71      7500

0.7104
original:
[[1554 2183]
 [ 447 3316]]
              precision    recall  f1-score   support

           0       0.78      0.42      0.54      3737
           1       0.60      0.88      0.72      3763

    accuracy                           0.65      7500
   macro avg       0.69      0.65      0.63      7500
weighted avg       0.69      0.65      0.63      7500

0.6493333333333333


In [19]:
print("KNN\n")

print("extended:")
print(confusion_matrix(y_test,predictionsc))
print(classification_report(y_test,predictionsc))
print(accuracy_score(y_test,predictionsc))

print("original:")
print(confusion_matrix(y_test1,predictions1c))
print(classification_report(y_test1,predictions1c))
print(accuracy_score(y_test1,predictions1c))

KNN

extended:
[[2360 1377]
 [1141 2622]]
              precision    recall  f1-score   support

           0       0.67      0.63      0.65      3737
           1       0.66      0.70      0.68      3763

    accuracy                           0.66      7500
   macro avg       0.66      0.66      0.66      7500
weighted avg       0.66      0.66      0.66      7500

0.6642666666666667
original:
[[2540 1197]
 [2259 1504]]
              precision    recall  f1-score   support

           0       0.53      0.68      0.60      3737
           1       0.56      0.40      0.47      3763

    accuracy                           0.54      7500
   macro avg       0.54      0.54      0.53      7500
weighted avg       0.54      0.54      0.53      7500

0.5392
